# Vector-space models: Static representations from contextual models

In [1]:
__author__ = "Christopher Potts"
__version__ = "CS224u, Stanford, Spring 2021"

## Contents

1. [Overview](#Overview)
1. [General set-up](#General-set-up)
1. [Loading Transformer models](#Loading-Transformer-models)
1. [The basics of tokenizing](#The-basics-of-tokenizing)
1. [The basics of representations](#The-basics-of-representations)
1. [The decontextualized approach](#The-decontextualized-approach)
  1. [Basic example](#Basic-example)
  1. [Creating a full VSM](#Creating-a-full-VSM)
1. [The aggregated approach](#The-aggregated-approach)
1. [Some related work](#Some-related-work)

## Overview



Can we get good static representations of words from models (like BERT) that supply only contextual representations? On the one hand, contextual models are very successful across a wide range of tasks, in large part because they are trained for a long time on a lot of data. This should be a boon for VSMs as we've designed them so far. On the other hand, the goal of having static representations might seem to be at odds with how these models process examples and represent examples. Part of the point is to obtain different representations for words depending on the context in which they occur, and a hallmark of the training procedure is that it processes sequences rather than individual words.

[Bommasani et al. (2020)](https://www.aclweb.org/anthology/2020.acl-main.431) make a significant step forward in our understanding of these issues. Ultimately, they arrive at a positive answer: excellent static word representations can be obtained from contextual models. They explore two strategies for achieving this:

1. __The decontextualized approach__: just process individual words as though they were isolated texts. Where a word consists of multiple tokens in the model, pool them with a function like mean or max.
1. __The aggregrated approach__: process lots and lots of texts containing the words of interest. As before, pool sub-word tokens, and also pool across all the pooled representations.

As Bommasani et al. say, the decontextualized approach "presents an unnatural input" – these models were not trained on individual words, but rather on longer sequences, so the individual words are infrequent kinds of inputs at best (and unattested as far as the model is concerned if the special boundary tokens [CLS] and [SEP] are not included). However, in practice, Bommasani et al. achieve very impressive results with this approach on word similarity/relatedness tasks.

The aggregrated approach is even better, but it requires more work and involves more decisions relating to which texts are processed.

This notebook briefly explores both of these approaches, with the goal of making it easy for you to apply these methods in [the associated homework and bakeoff](hw_wordrelatedness.ipynb).

## General set-up



In [8]:
import os
import pandas as pd
import numpy as np
import torch
from transformers import BertModel, BertTokenizer
from transformers import RobertaModel, RobertaTokenizer

import utils
import vsm

In [9]:
DATA_HOME = os.path.join('data', 'vsmdata')

In [10]:
utils.fix_random_seeds()

The `transformers` library does a lot of logging. To avoid ending up with a cluttered notebook, I am changing the logging level. You might want to skip this as you scale up to building production systems, since the logging is very good – it gives you a lot of insights into what the models and code are doing.

In [12]:
import logging
logger = logging.getLogger()
logger.level = logging.ERROR

## Loading Transformer models

To start, let's get a feel for the basic API that `transformers` provides. The first step is specifying the pretrained parameters we'll be using:

In [20]:
bert_weights_name = 'bert-base-uncased'

There are lots other options for pretrained weights. See [this Hugging Face directory](https://huggingface.co/models).

Next, we specify a tokenizer and a model that match both each other and our choice of pretrained weights:

In [21]:
bert_tokenizer = BertTokenizer.from_pretrained(bert_weights_name)

bert_model = BertModel.from_pretrained(bert_weights_name)

## The basics of tokenizing

It's illuminating to see what the tokenizer does to example texts:

In [23]:
example_text = "Bert knows Snuffleupagus"

Simple tokenization:

In [24]:
bert_tokenizer.tokenize(example_text)

['bert', 'knows', 's', '##nu', '##ffle', '##up', '##ag', '##us']

The `encode` method maps individual strings to indices into the underlying embedding used by the model:

In [25]:
ex_ids = bert_tokenizer.encode(example_text, add_special_tokens=True)

ex_ids

[101, 14324, 4282, 1055, 11231, 18142, 6279, 8490, 2271, 102]

We can get a better feel for what these representations are like by mapping the indices back to "words":

In [26]:
bert_tokenizer.convert_ids_to_tokens(ex_ids)

['[CLS]',
 'bert',
 'knows',
 's',
 '##nu',
 '##ffle',
 '##up',
 '##ag',
 '##us',
 '[SEP]']

Those are all the essential ingredients for working with these parameters in Hugging Face. Of course, the library has a lot of other functionality, but the above suffices for our current application.

## The basics of representations

To obtain the representations for a batch of examples, we use the `forward` method of the model, as follows:

In [27]:
with torch.no_grad():
    reps = bert_model(torch.tensor([ex_ids]), output_hidden_states=True)

In [36]:
for index, hs in enumerate(reps.hidden_states, 1):
    shape = hs.shape
    print(f'Hidden Shape {index}:    {shape}')

Hidden Shape 1:    torch.Size([1, 10, 768])
Hidden Shape 2:    torch.Size([1, 10, 768])
Hidden Shape 3:    torch.Size([1, 10, 768])
Hidden Shape 4:    torch.Size([1, 10, 768])
Hidden Shape 5:    torch.Size([1, 10, 768])
Hidden Shape 6:    torch.Size([1, 10, 768])
Hidden Shape 7:    torch.Size([1, 10, 768])
Hidden Shape 8:    torch.Size([1, 10, 768])
Hidden Shape 9:    torch.Size([1, 10, 768])
Hidden Shape 10:    torch.Size([1, 10, 768])
Hidden Shape 11:    torch.Size([1, 10, 768])
Hidden Shape 12:    torch.Size([1, 10, 768])
Hidden Shape 13:    torch.Size([1, 10, 768])


The return value `reps` is a special `transformers` class that holds a lot of representations. If we want just the final output representations for each token, we use `last_hidden_state`:

The shape indicates that our batch has 1 example, with 10 tokens, and each token is represented by a vector of dimensionality 768. 

Aside: Hugging Face `transformers` models also have a `pooler_output` value. For BERT, this corresponds to the output representation above the [CLS] token, which is often used as a summary representation for the entire sequence. However, __we cannot use `pooler_output` in the current context__, as `transformers` adds new randomized parameters on top of it, to facilitate fine-tuning. If we want the [CLS] representation, we need to use `reps.last_hidden_state[:, 0]`.

Finally, if we want access to the output representations from each layer of the model, we use `hidden_states`. This will be `None` unless we set `output_hidden_states=True` when using the `forward` method, as above. 

In [38]:
len(reps.hidden_states)

13

The length 13 corresponds to the initial embedding layer (layer 0) and the 12 layers of this BERT model.

The final layer in `hidden_states` is identical to `last_hidden_state`:

In [39]:
reps.hidden_states[-1].shape

torch.Size([1, 10, 768])

In [40]:
torch.equal(reps.hidden_states[-1], reps.last_hidden_state)

True

## The decontextualized approach

As discussed above, Bommasani et al. (2020) define and explore two general strategies for obtaining static representations for word using a model like BERT. The simpler one involves processing individual words and, where they correspond to multiple tokens, pooling those token representations into a single vector using an operation like mean.

### Basic example

To begin to see what this is like in practice, we'll use the method `vsm.hf_encode`, which maps texts to their ids, taking care to use `unk_token` for texts that can't otherwise be processed by the model.

Where a word corresponds to just one token in the vocabulary, it will get mapped to a single id:

In [43]:
token = bert_tokenizer.tokenize('puppy')

In [47]:
type(bert_tokenizer.encode(token))

list

In [48]:
vsm.hf_encode("puppy", bert_tokenizer)

tensor([[17022]])

As we saw above, some words map to multiple tokens:

In [49]:
bert_tokenizer.tokenize('snuffleupagus')

['s', '##nu', '##ffle', '##up', '##ag', '##us']

In [50]:
subtok_ids = vsm.hf_encode("snuffleupagus", bert_tokenizer)

subtok_ids

tensor([[ 1055, 11231, 18142,  6279,  8490,  2271]])

Next, the function `vsm.hf_represent` will map a batch of ids to their representations in a user-supplied model, at a specified layer in that model:

In [52]:
subtok_reps = vsm.hf_represent(subtok_ids, bert_model, layer=-1)

subtok_reps.shape

torch.Size([1, 6, 768])

The shape here: 1 example containing 6 (sub-word) tokens, each of dimension 768. With `layer=-1`, we obtain the final output repreentation from the entire model.

The final step is to pool together the 6 tokens. Here, we can use a variety of operations; [Bommasani et al. 2020](https://www.aclweb.org/anthology/2020.acl-main.431) find that `mean` is the best overall:

In [65]:
mean_tensor = torch.mean(subtok_reps, axis=1)
mean_tensor

tensor([[ 3.0305e-01,  1.2634e-01, -3.9157e-02,  2.0512e-01,  2.6558e-01,
         -1.2169e-01, -3.4225e-01,  4.1502e-01,  7.5906e-03, -5.5434e-01,
         -1.0319e-01, -3.4334e-01, -7.7310e-03,  1.3106e-01, -5.8204e-01,
          1.1481e+00, -5.0384e-01,  2.8346e-01,  1.4319e-01, -2.1976e-01,
          2.9822e-01, -2.7246e-01, -1.5947e-01,  1.5184e-01,  6.7372e-02,
          8.4873e-02,  5.6771e-01,  2.9787e-01, -1.9576e-01, -5.6074e-01,
          2.5558e-02,  2.9466e-01,  1.6190e-01, -4.2745e-02,  2.3723e-01,
          1.1799e-02,  2.4137e-01, -2.0017e-01,  8.8115e-02, -5.4736e-01,
         -5.7943e-01, -3.8881e-01, -2.0147e-01, -4.8178e-02, -1.5158e-01,
         -1.3919e-01,  1.7402e-01, -3.7898e-02,  2.4982e-01, -9.3894e-01,
         -7.7487e-01,  5.4588e-01,  5.8069e-01, -1.0230e-01, -2.4828e-02,
          3.5637e-01, -1.9554e-01, -5.5180e-01, -1.0707e-02,  1.3795e-01,
          1.9116e-01, -3.3160e-01,  1.0020e-01,  9.2630e-02, -2.2284e-01,
          5.5217e-01, -1.3917e-01,  2.

In [66]:
subtok_pooled = vsm.mean_pooling(subtok_reps)

subtok_pooled.shape

torch.Size([1, 768])

The function `vsm.mean_pooling` is simple `torch.mean` with `axis=1`. There are also predefined functions `vsm.max_pooling`, `vsm.min_pooling`, and `vsm.last_pooling` (representation for the final token).

### Creating a full VSM

Now we want to scale the above process to a large vocabulary, so that we can create a full VSM. The function `vsm.create_subword_pooling_vsm` makes this easy. To start, we get the vocabulary from one of our count VSMs (all of which have the same vocabulary):

In [78]:
vsm_index = pd.read_csv(
    os.path.join(DATA_HOME, 'yelp_window5-scaled.csv.gz'),
    usecols=[0], index_col=0)

In [79]:
vocab = list(vsm_index.index)

In [80]:
vocab[: 5]

['):', ');', '..', '...', ':(']

And then we use `vsm.create_subword_pooling_vsm`:

In [81]:
%%time
pooled_df = vsm.create_subword_pooling_vsm(
    vocab, bert_tokenizer, bert_model, layer=1)

CPU times: user 3min 3s, sys: 11.7 s, total: 3min 14s
Wall time: 3min 6s


The result, `pooled_df`, is a `pd.DataFrame` with its index given by `vocab`. This can be used directly in the word relatedness evaluations that are central the homework and associated bakeoff.

In [82]:
pooled_df.shape

(6000, 768)

In [90]:
vsm.neighbors('affordable', pooled_df)

affordable     0.000000
inexpensive    0.229579
cheaper        0.260831
priced         0.283825
expensive      0.299281
                 ...   
displeased     0.763204
purgatory      0.764573
gravy          0.777284
brunch         0.779698
schnauzer      0.842928
Length: 6000, dtype: float64

This approach, and the associated code, should work generally for all Hugging Face Transformer-based models. Bommasani et al. (2020) provide a lot of guidance when it comes to how the model, the layer choice, and the pooling function interact.

## The aggregated approach

The aggregated is also straightfoward to implement given the above tool. To start, we can create a map from vocabulary items into their sequences of ids:

In [91]:
vocab_ids = {w: vsm.hf_encode(w, bert_tokenizer)[0] for w in vocab}

Next, let's assume we have a corpus of texts that contain the words of interest:

In [95]:
corpus = [
    "This is a sailing example",
    "It's fun to go sailing!",
    "We should go sailing.",
    "I'd like to go sailing and sailing",
    "This is merely an example"]

The following embeds every corpus example, keeping `layer=1` representations:

In [97]:
corpus_ids = [vsm.hf_encode(text, bert_tokenizer)
              for text in corpus]

In [98]:
corpus_reps = [vsm.hf_represent(ids, bert_model, layer=1)
               for ids in corpus_ids]

Finally, we define a convenience function for finding all the occurrences of a sublist in a larger list:

In [99]:
def find_sublist_indices(sublist, mainlist):
    indices = []
    length = len(sublist)
    for i in range(0, len(mainlist)-length+1):
        if mainlist[i: i+length] == sublist:
            indices.append((i, i+length))
    return indices

For example:

In [100]:
find_sublist_indices([1,2], [1, 2, 3, 0, 1, 2, 3])

[(0, 2), (4, 6)]

And here's an example using our `vocab_ids` and `corpus`:

In [101]:
sailing = vocab_ids['sailing']

In [103]:
sailing_reps = []

for ids, reps in zip(corpus_ids, corpus_reps):
    offsets = find_sublist_indices(sailing, ids.squeeze(0))
    for (start, end) in offsets:
        pooled = vsm.mean_pooling(reps[:, start: end])
        sailing_reps.append(pooled)

sailing_rep = torch.mean(torch.cat(sailing_reps), axis=0).squeeze(0)

In [104]:
sailing_rep.shape

torch.Size([768])

The above building blocks could be used as the basis for an original system and bakeoff entry for this unit. The major question is probably which data to use for the corpus.

## Some related work

1. [Ethayarajh (2019)](https://www.aclweb.org/anthology/D19-1006/) uses dimensionality reduction techniques (akin to LSA) to derive static representations from contextual models, and explores layer-wise variation in detailed, with findings that are likely to align with your experiences using the above techniques.

1. [Akbik et al (2019)](https://www.aclweb.org/anthology/N19-1078/) explore techniques similar to those of Bommasani et al. specifically for the supervised task of named entity recognition.

1. [Wang et al. (2020](https://arxiv.org/pdf/1911.02929.pdf) learn static representations from contextual ones using techniques adapted from the word2vec model.